<a href="https://colab.research.google.com/github/Priyo-prog/Deep-Learning-with-Tensorflow/blob/main/Natural%20Language%20Processing/NLP_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Introduction

NLP has the goal of derieving information out of natural language (could be sequence text or speech).

Another common terms for NL problems is sequence to sequence problems (seq2seq).

## Check For GPU

In [1]:
!nvidia-smi

Tue Nov 12 01:03:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Get Helper Functions

In [2]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-11-12 01:03:11--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-11-12 01:03:12 (89.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a Text Dataset

Thedataset we are going to use is Kaggle's intoduction to NLP datset (testx sample of tweets labelled as disaster or not disaster) Binary Classification

In [4]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-11-12 01:03:24--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2024-11-12 01:03:24 (102 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Import Libraries

In [5]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

## Visualizing Text Dataset

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [8]:
# Shuffle the training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [10]:
# Let's visualize some random training samples
# create random indexes not higher than the total number of samples
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  # First underscore is to skip the index number
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n {text}\n")
  print("-----\n")

Target: 0 (not real disaster)
Text:
 Get å£150 free bets when you register at Boylesports Casino #Luck #Destroy http://t.co/zildpvKNXP http://t.co/5yDb4s13pF

-----

Target: 0 (not real disaster)
Text:
 @CaraJDeIevingnc the bomb impact ratio hit beyond kyle js

-----

Target: 1 (real disaster)
Text:
 Nashville Theater Attack: Will Gun Grabbers Now Demand ÛÏHatchet Control?Û  http://t.co/OyoGII97yH

-----

Target: 0 (not real disaster)
Text:
 @jasminehuerta24 I hope you drown ??

-----

Target: 1 (real disaster)
Text:
 Another white guy trying to mass murder people for no apparent reason just because let me guess he's mentally ill blah blah blah #Antioch

-----



## Split Data Into Training And Validation Sets

In [11]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(), train_df_shuffled["target"].to_numpy(),
    test_size=0.1, random_state=42)

In [12]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [13]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Change Text to Numbers

**Tokenization** - Straight mapping from token(a token could be aword or character) to number (can be modelled but quickly gets too big). Can be direct mapping like `0,1,2` or `A-1, B-2.... Z-26` or `one hot encoding` in that case it would be too large to handle

**Embedding** - Richer representation of relationships between tokens (can limit size + can be learned). This can update at each epochs of deep neural network learning.

In [14]:
# Use the Default Text Vectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create gropus of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    pad_to_max_tokens=False)

## Average number of tokens in training tweets

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

## Setup Text Vectorization Variables

In [16]:
max_vocab_length = 10000 # max number of vocab to have in our vocabulary
max_length = 15 # max length our sequences will be(e.g. how many words from a tweet does a model see ?)
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [18]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original Sentence \n {random_sentence}\
      \n\nVectorized Version: ")
text_vectorizer([random_sentence])

Original Sentence 
 I've been meaning to harm you in the best way I see fit??      

Vectorized Version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 276,   59, 5150,    5,  396,   12,    4,    2,  149,  147,    8,
          99, 3845,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique words in the vocabulary
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab : {len(words_in_vocab)}")
print(f"5 most common words : {top_5_words}")
print(f"5 least common words : {bottom_5_words}")

Number of words in vocab : 10000
5 most common words : ['', '[UNK]', 'the', 'a', 'in']
5 least common words : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Create Embedding Using Embedding Layer

To make our embedding we are going to use Tensorflow's embedding layer : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about our embedding layer:
* `input_dim` - The size of our vocabulary
* `output_dim` - the size of the output embedding vector, for example, a value of 100 would mean each tokens gets represented by vector 100 long

* `input_length` - Length of the sequences being passed to the embedding layer

In [21]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, #output_shape
                             input_length=max_length, # how long is each input
                             )
embedding

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding, built=False>

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original Text :\n{random_sentence}\
       \n\n Embedded version : ")

# Embded the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text :
A whistleblower lawsuit accuses that supervisor of sleeping on the job more than once according to officials. 

http://t.co/IPwySnik0G       

 Embedded version : 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04213457,  0.04828483,  0.04253319, ...,  0.01180469,
          0.04529662, -0.04366568],
        [-0.02784282,  0.04173216,  0.01544284, ..., -0.04458594,
         -0.02051488,  0.0339952 ],
        [-0.00644193, -0.04115602, -0.03741568, ..., -0.04991143,
         -0.03987157,  0.04101625],
        ...,
        [ 0.0391405 , -0.02185716, -0.00980805, ..., -0.02283149,
          0.0125044 , -0.04126489],
        [ 0.00276799,  0.02029195, -0.00954844, ...,  0.04933205,
          0.01163449, -0.00407913],
        [ 0.03751642, -0.04645863,  0.04323344, ...,  0.01157224,
         -0.00546537,  0.03832643]]], dtype=float32)>

In [23]:
# Check out single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04213457,  0.04828483,  0.04253319, -0.00746244, -0.03965837,
         0.02190801, -0.03651488, -0.02689352, -0.04253178, -0.02609805,
        -0.00241914,  0.02374181,  0.03213269, -0.02902921, -0.00459992,
        -0.03019511, -0.01964537,  0.00155628,  0.0006777 ,  0.02616891,
         0.0220495 , -0.00436914,  0.02124805,  0.01373399, -0.03985786,
        -0.02161267, -0.01029711,  0.02798973, -0.0115549 ,  0.04816157,
         0.04265541,  0.01727043,  0.03799082, -0.00666178,  0.00966407,
         0.02930199, -0.01232474, -0.02533221,  0.03451291, -0.03110815,
         0.04971245,  0.04372028,  0.03195889, -0.01128763, -0.02022111,
         0.03032557, -0.02963811, -0.01013712,  0.00367548, -0.00734661,
         0.04658492, -0.0342731 ,  0.04934042,  0.0409121 , -0.01606978,
        -0.02038487, -0.03962228,  0.02613996,  0.03911998, -0.04893131,
        -0.02597694, -0.04765047,  0.01796193,  0.00306837, -0.04925178,
  

## Modelling a dataset (running a set of experiments)

Now we've a got way to turn out text sequences into numbers it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn-map
* Model 1: Feed-Forward Neural Network (Dense Model)
* Model 2: LSTM model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

**Approach :**

* Create a model
* Build a model
* Fit a model
* Evaluate our model




## Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create baseline model so you've got a benchmark for future experiments to build upon.

To crate our baseline, we'll use Sklearn's Multinomial Naive Bayes using TF-IDF Formula to convert our words to numbers.

**Note** : It's common use to non-DL algorithms as a baseline because of their speed and then later using DL to see if you improve upon them.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create Tokenization and modelling pipeline
model_0 = Pipeline(
    steps=[("ifidf", TfidfVectorizer()), # convert numbers into words using tfidf
           ("clf", MultinomialNB()) # Model The text
           ])

# Fit The pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('ifidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
## Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating Evaluation Function for Model Experiments

In [28]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
   Calculates model accuracy, recall, precision and f1 score
   for a binary classification model
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [29]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [30]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

## Create Model With Functional API

In [31]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # Inputs are 1-dimensional String
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)  # Flatten the embedding output
outputs = layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(x) # Create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [32]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# compile the model
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [35]:
# fit the model
model_1_history = model_1.fit(x=train_sentences,
                               y=train_labels,
                               epochs=5,
                              validation_data=(val_sentences, val_labels),
                               callbacks=[create_tensorboard_callback(dir_name="SAVE_DIR",
                                                                      experiment_name="model_1_dense")])

Saving TensorBoard log files to: SAVE_DIR/model_1_dense/20241112-010332
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.6442 - loss: 0.6512 - val_accuracy: 0.7743 - val_loss: 0.5350
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8087 - loss: 0.4699 - val_accuracy: 0.7913 - val_loss: 0.4735
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8617 - loss: 0.3553 - val_accuracy: 0.7900 - val_loss: 0.4584
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8854 - loss: 0.2957 - val_accuracy: 0.7848 - val_loss: 0.4702
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9105 - loss: 0.2413 - val_accuracy: 0.7900 - val_loss: 0.4818


## Check The Model Predictions

In [36]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7818 - loss: 0.5155


[0.48184332251548767, 0.7900262475013733]

In [37]:
# make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


(762, 1)

If we didn't put the GlobalAveragePooling1D for flatenning the layer then prediction would have come for all the tokens rather than the entire sample sentence

In [38]:
model_1_pred_probs[0]

array([0.32510954], dtype=float32)

In [39]:
# Look at The first 10 predictions
model_1_pred_probs[:10]

array([[0.32510954],
       [0.73063564],
       [0.9976246 ],
       [0.10109074],
       [0.09824141],
       [0.9340345 ],
       [0.9251921 ],
       [0.99136883],
       [0.9617882 ],
       [0.23742111]], dtype=float32)

## Compare Predictions With Labels

In [40]:
 # Convert model prediction probabilities to label format
 model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
 model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

## Classification result comparison of Labels and Predictions

In [41]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.00262467191601,
 'precision': 0.7969833282849871,
 'recall': 0.7900262467191601,
 'f1': 0.786470332384797}

In [42]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False,  True])

In [43]:
 baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Visualizing The Learned Embeddings

In [44]:
## Get the vocabulary from the text vectorization layer
word_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Get the weight matrix of the embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for ~5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)

Now We've got the embedding matrix our model has learnt to represent our tokens, let's see how we can visualize it.

To do so tensorflow has a handy tool called projector:

And Tensorflow also has an incredible guide on word embeddings themselves :
http://projector.tensorflow.org/

https://www.tensorflow.org/text/guide/word_embeddings


In [46]:
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [47]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN)

RNN's are useful for sequence data

The premise of a recurrent neural network is to use the  representation of previous input to aid the  representation of a later input.


**Resources**
- MIT 's sequence modelling lecture
https://www.youtube.com/watch?v=QvkQ1B3FBqA

- Chris Olah's intro to LSTMs:
https://colah.github.io/posts/2015-08-Understanding-LSTMs/

- Anderj Karpathy's the unreasonable effectiveness of recurrent neural networks :

https://karpathy.github.io/2015/05/21/rnn-effectiveness/

## Model 2: LSTM

LSTM = Long Short Term Memory (one of the most popular LSTM cells)

Our structure f an RNN looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers(RNN/Dense) -> Output (label probability)

````

In [48]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(units=64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to set return_sequences=True
print(x.shape)
x = layers.LSTM(units=64)(x)
print(x.shape)
x = layers.Dense(units=64, activation=tf.keras.activations.relu)(x)
outputs = layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_lstm")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [49]:
# model summary
model_2.summary()

Model: "model_2_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 15, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,366,657 (5.21 MB)

 Trainable params: 1,366,657 (5.21 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# compile the model
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [51]:
# Now fit the model
model_2.history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20241112-010402
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9113 - loss: 0.2865 - val_accuracy: 0.7822 - val_loss: 0.5673
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9390 - loss: 0.1598 - val_accuracy: 0.7795 - val_loss: 0.7020
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9468 - loss: 0.1387 - val_accuracy: 0.7848 - val_loss: 0.6231
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9593 - loss: 0.1099 - val_accuracy: 0.7835 - val_loss: 0.8030
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9623 - loss: 0.0929 - val_accuracy: 0.7769 - val_loss: 1.2220


## Make Prediction With LSTM Model

In [52]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


array([[4.2168954e-03],
       [8.5772938e-01],
       [9.9998498e-01],
       [8.3643170e-03],
       [1.7374050e-04],
       [9.9993563e-01],
       [9.9841356e-01],
       [9.9998915e-01],
       [9.9998224e-01],
       [4.4782808e-01]], dtype=float32)

## Convert the Model 2 Prediction to Labels

In [53]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [54]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.69028871391076,
 'precision': 0.7775711689884918,
 'recall': 0.7769028871391076,
 'f1': 0.7754662769426549}

In [55]:
 baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 3 : GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cells has similar features to LSTM cell but  has less parameters.

In [56]:
from tensorflow.keras import layers

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each others, you need return_sequences=True
print(x.shape)
x = layers.LSTM(units=64, return_sequences=True)(x)
print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x)
print(x.shape)
x = layers.Dense(units=64, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(units=1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


(None, 15, 128)
(None, 15, 64)
(None, 15, 64)
(None, 15, 64)


In [57]:
# Get the summary
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 15, 64)              │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 15, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 15, 64)              │          24,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 15, 64)              │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,379,457 (5.26 MB)

 Trainable params: 1,379,457 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
# compile the model
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# fit the model
model_3.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                   "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20241112-010419
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9032 - loss: 0.2530 - val_accuracy: 0.7769 - val_loss: 0.7166
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9664 - loss: 0.0869 - val_accuracy: 0.7769 - val_loss: 0.9316
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9722 - loss: 0.0672 - val_accuracy: 0.7612 - val_loss: 1.0362
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9666 - loss: 0.0802 - val_accuracy: 0.7743 - val_loss: 1.4977
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9734 - loss: 0.0596 - val_accuracy: 0.7703 - val_loss: 1.5170


## Make Prediction With GRU Model

In [59]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


array([[2.1804508e-04],
       [9.4636554e-01],
       [9.9999273e-01],
       [1.1236713e-01],
       [4.1709941e-06],
       [9.9998176e-01],
       [9.9952912e-01],
       [1.0000000e+00],
       [9.9999964e-01],
       [4.6431246e-01]], dtype=float32)

## Convert the Model 3 Prediction to Labels

In [60]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [61]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.03412073490814,
 'precision': 0.7720778836459699,
 'recall': 0.7703412073490814,
 'f1': 0.7681933440908485}

## Model 4 : Bidirectional RNN Model

`Standard RNN`
In a typical RNN, the model processes sequential data by maintaining a hidden state that captures information from the past (previous time steps) as it moves from the beginning to the end of a sequence. However, it only considers the previous (past) context, which may not be ideal in tasks where the future context also plays an important role.

`Bidirectional RNN`
A Bidirectional RNN addresses this limitation by processing the sequence data in both directions:

Forward RNN: Processes the sequence from the first time step to the last (like a standard RNN).
Backward RNN: Processes the sequence from the last time step to the first (in reverse order).

In [62]:
# Build a Bidirectional RNN in Tensorflow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(units=1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

(None, 15, 128)


In [63]:
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 15, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,453,441 (5.54 MB)

 Trainable params: 1,453,441 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [64]:
# compile the model 4
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20241112-010439
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9600 - loss: 0.1740 - val_accuracy: 0.7638 - val_loss: 0.9147
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9767 - loss: 0.0561 - val_accuracy: 0.7572 - val_loss: 1.0373
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9740 - loss: 0.0550 - val_accuracy: 0.7651 - val_loss: 1.3338
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9781 - loss: 0.0434 - val_accuracy: 0.7743 - val_loss: 1.2527
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9781 - loss: 0.0454 - val_accuracy: 0.7664 - val_loss: 1.1804


## Check The Model 4 Prediction

In [65]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


array([[3.7259667e-03],
       [7.6930553e-01],
       [9.9980050e-01],
       [1.9366463e-01],
       [3.4457738e-05],
       [9.9946111e-01],
       [8.7247479e-01],
       [9.9995756e-01],
       [9.9989760e-01],
       [6.7006272e-01]], dtype=float32)

## Compare Prediction Model 4 With Labels

In [66]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [67]:
# Calculate the results for our bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 76.64041994750657,
 'precision': 0.769346188182321,
 'recall': 0.7664041994750657,
 'f1': 0.7635833452096769}

In [68]:
model_3_results

{'accuracy': 77.03412073490814,
 'precision': 0.7720778836459699,
 'recall': 0.7703412073490814,
 'f1': 0.7681933440908485}

Model 3 is performing better

## Convolutional Network for Text

We've used CNNs for images but images are typically 2D (height x width).. however our text data is 1D.

```
 Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D  + pooling) -> Outputs(class probabilities)
 ```

## Model 5 : Conv1D

For different explanations of parametres see:
* https://poloclub.github.io/cnn-explainer/

* Difference between `same` and `valid` padding

In [69]:
from tensorflow.keras import layers
# Test our embedding layer , Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is test sentence"])) # turn target sequences into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        strides=1, #default
                        activation="relu",
                        padding="valid")
conv_1d_output = conv_1d(embedding_test) # pass embedding through conv1d layer
max_pool = layers.MaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the features with highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 5, 32]))

In [70]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04973917, -0.01109379,  0.00816894, ...,  0.00688494,
          0.00153654,  0.0107775 ],
        [ 0.01156143, -0.04611303,  0.00805117, ...,  0.05405089,
         -0.0802077 , -0.02866453],
        [-0.0240022 , -0.03922223, -0.05726291, ..., -0.03557978,
         -0.02042248, -0.01165623],
        ...,
        [-0.02518125, -0.00562928, -0.01919252, ..., -0.01470093,
         -0.03951335, -0.02808012],
        [-0.02518125, -0.00562928, -0.01919252, ..., -0.01470093,
         -0.03951335, -0.02808012],
        [-0.02518125, -0.00562928, -0.01919252, ..., -0.01470093,
         -0.03951335, -0.02808012]]], dtype=float32)>

In [71]:
 conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.03246848, 0.00127651, 0.02050976, 0.        , 0.00156233,
         0.067067  , 0.10975052, 0.06590514, 0.        , 0.01699849,
         0.        , 0.        , 0.        , 0.00530268, 0.        ,
         0.0014647 , 0.03729342, 0.        , 0.        , 0.        ,
         0.09762505, 0.        , 0.        , 0.        , 0.        ,
         0.04199247, 0.        , 0.        , 0.02685106, 0.        ,
         0.03368823, 0.        ],
        [0.        , 0.        , 0.        , 0.03148132, 0.02380265,
         0.01169014, 0.06609628, 0.        , 0.09236816, 0.01575923,
         0.        , 0.        , 0.        , 0.03129622, 0.03032221,
         0.0561124 , 0.        , 0.        , 0.01929568, 0.        ,
         0.02895031, 0.02252838, 0.        , 0.04532235, 0.        ,
         0.0750213 , 0.017955  , 0.01108092, 0.06048813, 0.04108138,
         0.0979002 , 0.        ],
        [0.        , 0.06076831, 0.        , 0.0203

In [72]:
# Create 1D Convolutional Neural Network to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, activation="relu",
                   strides=1, padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(units=1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv_1d")

# Model compile
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get the summary of conv_1d model
model_5.summary()

Model: "model_5_conv_1d"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 11, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,321,089 (5.04 MB)

 Trainable params: 1,321,089 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [73]:
 # Fit The Model
 model_5.fit(train_sentences,
             train_labels,
             epochs=5,
             validation_data=(val_sentences, val_labels),
             callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                    "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20241112-010500
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9355 - loss: 0.1854 - val_accuracy: 0.7769 - val_loss: 0.8617
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9778 - loss: 0.0705 - val_accuracy: 0.7598 - val_loss: 0.9939
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9786 - loss: 0.0585 - val_accuracy: 0.7612 - val_loss: 1.1250
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9787 - loss: 0.0510 - val_accuracy: 0.7572 - val_loss: 1.1215
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9774 - loss: 0.0552 - val_accuracy: 0.7533 - val_loss: 1.2105


## Model prediction with Conv 1D model

In [74]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


array([[3.9263222e-01],
       [9.6273428e-01],
       [9.9994266e-01],
       [3.3368416e-02],
       [1.1298545e-05],
       [9.9949670e-01],
       [9.9733472e-01],
       [9.9997938e-01],
       [9.9999928e-01],
       [9.3079013e-01]], dtype=float32)

In [75]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [76]:
# Evaluate model 5 predictictions
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 75.32808398950131,
 'precision': 0.7533943930774577,
 'recall': 0.7532808398950132,
 'f1': 0.7518108425860833}

## Model 6: Tensorflow Hub Pretrained Sentence Encoder

USE -> Universal Sentence Encoder

In [77]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                       "When you can the universal sentence encoder on a sentence, it turns it into numbers"])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [78]:
embed_samples[0].shape

TensorShape([512])

## Create Keras Layer using USE

Create a keras layer using USE pretrained layer from tensorflow hub

In [79]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # can be of variable length, that is why it is empty
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [80]:
import tf_keras

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

model_6 = tf_keras.Sequential([
  sentence_encoder_layer,
  #tf_keras.layers.Dense(256, activation="relu"), # Use tf_keras.layers.Dense
  #tf_keras.layers.Dense(128, activation="relu"), # Use tf_keras.layers.Dense
  tf_keras.layers.Dense(64, activation="relu"), # Use tf_keras.layers.Dense
  tf_keras.layers.Dense(1, activation="sigmoid", name="output_layer") # Use tf_keras.layers.Dense
], name="model_6_USE")

model_6.compile(loss="binary_crossentropy",
                optimizer=tf_keras.optimizers.Adam(),
                metrics=["accuracy"])
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [81]:
model_6_history = model_6.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 7s 13ms/step - loss: 0.5104 - accuracy: 0.7784 - val_loss: 0.4474 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4132 - accuracy: 0.8167 - val_loss: 0.4351 - val_accuracy: 0.8110
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4000 - accuracy: 0.8213 - val_loss: 0.4300 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3916 - accuracy: 0.8263 - val_loss: 0.4282 - val_accuracy: 0.8123
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3840 - accuracy: 0.8330 - val_loss: 0.4264 - val_accuracy: 0.8150


USE TF : Make Predictions

In [82]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 9ms/step


array([[0.14021477],
       [0.7850791 ],
       [0.9930226 ],
       [0.20231792],
       [0.71967745],
       [0.62051785],
       [0.9864748 ],
       [0.97708124],
       [0.9397078 ],
       [0.09536614]], dtype=float32)

In [83]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [84]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 81.49606299212599,
 'precision': 0.8182618558292442,
 'recall': 0.8149606299212598,
 'f1': 0.8131230163581015}

In [86]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 7: Pretrained USE but 10% Training Data

Transfer learning helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model 6` except we will train it on 10% of the data.